In [411]:
import pandas as pd

def extract_row(row):
    transactions = []
    for s, d in [(f"s{i}",f"d{i}") for i in range(1,6)]:
        transactions.append((row["Date"], row[s], row[d]))
    return transactions


def find_last_update_date(df):
    return df[df["s1"].notna()]["Date"]

    
def spends_on_date(df,dat):
    return df[(df["Date"] == dat) & df["s1"].notnull() & (df["s1"] > 0)]


def find_spend_on_date(date):
    for name, df in sheet_to_df_map.items():
        d = spends_on_date(df, date)
        if not d.empty:  
            d = d.iloc[0]
            print("="*80)
            print(name, d.dropna())
            print("="*80)
            

cols = ["Date", "balance", 's1', 'd1', 's2', 'd2', 's3', 'd3', 's4', 'd4', 's5', 'd5', 's6', 'd6', 's7', 'd7'] 
xls = pd.ExcelFile('./source.xlsx')
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    if sheet_name in ["Sheet1", "Аня ДР", "На дом"]:
        continue
    df = xls.parse(sheet_name, parse_dates=True, names=cols, header=None)
    df = df.drop(0)
    df = df.drop('balance', axis=1)
    sheet_to_df_map[sheet_name] = df
    
fund_names = list(sheet_to_df_map.keys())

for name in fund_names: 
    df = sheet_to_df_map[name]
    print(name, (df.shape[1]))

Еда 15
Домхоз 15
Подарки 15
Кот 15
Развлечения 15
Конь 15
Машина 15
Медицина 15
Отбираловка 15
Собака 15
Бизнес 15
Хобби 15


# Prompt to Codeium

I have a dataset with columns ["Date", "balance", 's1', 'd1', 's2', 'd2', 's3', 'd3', 's4', 'd4', 's5', 'd5', 's6', 'd6', 's7', 'd7']. Each row in dataset is a transactions up to 7 of them on same date. Date column is a date of transactions. Columns "sN" (N means number fom 1 to 7) is a transaction amount. Columns "dN" (N means number from 1 to 7) is a transaction description. So, is "s1" contains a number on some row, it means that on this row and it's date there was 1 transaction with amount in "s1" and description on "d1". If there "s2" number present - it's a second transaction on same date with description in "d2". Write a code that split such grouping to new dataset with columns ["amount", "date", "description"]. If you meet a row with s1 filled and s2 filled, then you should create 2 rows in new dataset with same date, where first row will be filled by this columns mapping "Date" -> "date", "s1" -> "amount", "d1" -> "description" and second row which will be filled by this columns mapping "Date" -> "date", "s2" -> "amount", "d2" -> "description". Describe step by step why you wrote each code line.

In [412]:
def convert_to_new(df):
    # Create a list to store the transformed data
    transformed_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Iterate over the transactions (pair of amount and description)
        for i in range(1, 8):
            amount_col = f"s{i}"
            desc_col = f"d{i}"
            # Check if the amount is present in the row
            if pd.notnull(row[amount_col]):
                # Append a new row to the transformed data list with the amount, date, and description
                transformed_data.append({
                    "amount": row[amount_col],
                    "date": row["Date"].strftime('%d.%m.%Y'),
                    "description": row[desc_col],
                    "synced": True
                })

    # Create a new DataFrame from the transformed data list
    return pd.DataFrame(transformed_data)

0     2023-02-01
1     2023-02-17
2     2023-02-17
3     2023-02-19
4     2023-02-19
         ...    
131   2024-01-03
132   2024-01-03
133   2024-01-03
134   2024-01-05
135   2024-01-06
Name: date, Length: 136, dtype: datetime64[ns]

In [335]:
find_spend_on_date("2023-9-3")

In [413]:
new_dfs = {}
for name, df in sheet_to_df_map.items():
    new_dfs[name] = convert_to_new(df)

In [414]:
funds_df = pd.DataFrame([
    {"name": name, "budget": 0, "balance": f'=SUM(INDIRECT($A{idx+2}&"!$A{idx+2}:$A"))', "needSync": f'=NOT(XLOOKUP(FALSE;INDIRECT($A{idx+2}&"!$D:$D");INDIRECT($A{idx+2}&"!$D:$D");TRUE))'} for idx,name in enumerate(fund_names)
])

print(funds_df[["name", "balance"]])

           name                          balance
0           Еда    =SUM(INDIRECT($A2&"!$A2:$A"))
1        Домхоз    =SUM(INDIRECT($A3&"!$A3:$A"))
2       Подарки    =SUM(INDIRECT($A4&"!$A4:$A"))
3           Кот    =SUM(INDIRECT($A5&"!$A5:$A"))
4   Развлечения    =SUM(INDIRECT($A6&"!$A6:$A"))
5          Конь    =SUM(INDIRECT($A7&"!$A7:$A"))
6        Машина    =SUM(INDIRECT($A8&"!$A8:$A"))
7      Медицина    =SUM(INDIRECT($A9&"!$A9:$A"))
8   Отбираловка  =SUM(INDIRECT($A10&"!$A10:$A"))
9        Собака  =SUM(INDIRECT($A11&"!$A11:$A"))
10       Бизнес  =SUM(INDIRECT($A12&"!$A12:$A"))
11        Хобби  =SUM(INDIRECT($A13&"!$A13:$A"))


In [415]:
#
with pd.ExcelWriter('output_file.xlsx') as writer:
    for name, df in new_dfs.items():
        df.to_excel(writer, sheet_name=name, index=False)
    funds_df.to_excel(writer,sheet_name='Funds', index=False)

,Date,s1,d1,s2,d2,s3,d3,s4,d4,s5,d5,s6,d6,s7,d7
11,2023-02-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2023-02-11 00:00:00,1452.20,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2023-02-12 00:00:00,620.45,вв,2069.00,"орехи, семечки",1899.00,молоко,1512.0,вода,NaN,NaN,NaN,NaN,NaN,NaN
14,2023-02-13 00:00:00,858.80,вв,95.98,вода,666.75,магнит,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2023-02-14 00:00:00,1400.00,орехи,1300.00,рынок,2835.10,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2023-02-15 00:00:00,1091.52,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2023-02-16 00:00:00,270.90,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2023-02-17 00:00:00,1194.90,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2023-02-18 00:00:00,1035.00,вв,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2023-02-19 00:00:00,1756.11,вв,125.10,вв,450.00,яйки,4900.0,рынок,NaN,NaN,NaN,NaN,NaN,NaN
